# Installing Dependecies

In [1]:
%pip install gpt-2-simple

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.8.1-py3-none-any.whl size=24576 sha256=e279c39f4e274daebc78ade76beeacfe4132aa1915fc399885ff0674cafe18c3
  Stored in directory: /root/.cache/pip/wheels/65/71/09/4761ea784b90a1ce6f1ab07c6eb553a35ce75bd23350bcf147
Successfully built gpt-2-simple
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-be

# Import of dependencies

In [5]:
import gpt_2_simple as gpt2
import tensorflow as tf
import pandas as pd

# Creating Session, Closing Session, restore session to aviod OME

In [6]:
def CreateSession():
    sess = gpt2.start_tf_sess(threads=4)
    gpt2.load_gpt2(sess, run_name='/kaggle/input/notebook867863cc1c/checkpoint/run1')
    return sess


def closeSession(sess):
    gpt2.reset_session(sess)


def Restore_Session(sess):
    gpt2.reset_session(sess)
    sess = gpt2.start_tf_sess(threads=4)
    gpt2.load_gpt2(sess,run_name='/kaggle/input/notebook867863cc1c/checkpoint/run1')
    return sess


# Creating a output checkpointing directory for testing Outputs 

In [7]:
import os
# testCheckPoints
dir = "testCheckPoints"
if not os.path.exists(dir):
    os.mkdir(dir)

# Running on 5 Examples 

In [8]:
# closeSession(sess)
file_path= "/kaggle/input/notebook867863cc1c/testReview.csv"
df = pd.read_csv(file_path, header=None)
# delete the zeroth row with null values
df.columns = ['index', 'text', 'sentiments']
data = df.drop([0])

start=5
end  =20
sess= CreateSession()
review, predictedReview, sentiment, ActualSentiment = [], [], [], []
for index, row in data.iterrows():
    if index<start:
        continue
    if len(row['text'].split())<460:              
        testStr= "REVIEW_ {} ".format(row['text'])
        text = gpt2.generate(sess,
                            length=len(testStr.split()),
                            temperature=0.7,
                            prefix=testStr,
                            truncate="_END_",
                            batch_size=1,
                            checkpoint_dir="/kaggle/input/notebook867863cc1c/checkpoint",
                            nsamples=1,
                            return_as_list=True
                )


        # split the text to get the predicted review sentiment
        splitedText = text[0].split("_SENTIMENTS_")
        if len(splitedText) > 1:
            review.append(row['text'])
            sentiment.append(row['sentiments'])
            predictedReview.append(splitedText[0])
            ActualSentiment.append(splitedText[1])


        if index%1==0:
            print("Processing completed: ", index)

    if index==end:
        break 
          

closeSession(sess)




Loading checkpoint /kaggle/input/notebook867863cc1c/checkpoint/run1/model-41000
Processing completed:  5
Processing completed:  7
Processing completed:  8
Processing completed:  9
Processing completed:  10
Processing completed:  11
Processing completed:  12
Processing completed:  13
Processing completed:  15
Processing completed:  16
Processing completed:  17
Processing completed:  18
Processing completed:  19
Processing completed:  20


In [10]:
review, predictedReview, sentiment, ActualSentiment 
for index in range(len(review)):
    print("Review 0  :: ", review[index])
    print("Sentiment :: ", sentiment[index] )
    print("Predicted :: ", ActualSentiment[index])
    print("---------------------------------------------------------------------------------------------")
    

Review 0  ::  A true wholesome American story about teenagers who are interested in launching their own rocket in a rural West Virginia coal mining town, after the launch of Sputnik in 1957.<br /><br />Through trial, tribulations and perseverance beyond belief, they are ultimately able to achieve their goals.<br /><br />Jake Gyllenhaal, as the leader of the group, is excellent in the title role. As his motivating science teacher, Laura Linney is quite good but her southern accent is over the top.<br /><br />There is a standout supporting performance by Chris Cooper, a head miner, who wants his son to follow in his footsteps, but gradually comes around at film's end.<br /><br />What makes this film so unusual for our times is that there are no bed-hopping scenes and no profanity whatsoever. It is the epitome of an American story that is well done.<br /><br />Besides the science angle, we have the father-son disagreement, football scholarships as a way to escape coal mining, and the lovi

# Running on test dataset and Saving the Output

In [ ]:
start1=0
start=start1+1
end  =start1+313
def RunOnDataset(data):
    sess= CreateSession()
    review, predictedReview, sentiment, ActualSentiment = [], [], [], []
    try:
        for index, row in data.iterrows():
            if index<start:
                continue
            if len(row['text'].split())<460:              
                testStr= "REVIEW_ {} ".format(row['text'])
                text = gpt2.generate(sess,
                                    length=len(testStr.split()),
                                    temperature=0.7,
                                    prefix=testStr,
                                    truncate="_END_",
                                    batch_size=1,
                                    checkpoint_dir="/kaggle/input/notebook867863cc1c/checkpoint",
                                    nsamples=1,
                                    return_as_list=True
                                    )


                # split the text to get the predicted review sentiment
                splitedText = text[0].split("_SENTIMENTS_")
                if len(splitedText) > 1:
                    review.append(row['text'])
                    sentiment.append(row['sentiments'])
                    predictedReview.append(splitedText[0])
                    ActualSentiment.append(splitedText[1])
                
            if index!=0: 
                if index%3== 0:
                    print("Processing completed: ", index)
            
                if index%50==0:
                    # Save the outputs to csv 
                    outputFile= "testCheckPoints/output_{}.csv".format(index)
                    df = pd.DataFrame({ 'review': review, 'predictedReview': predictedReview, 'sentiment': sentiment, 'ActualSentiment': ActualSentiment})
                    df.to_csv(outputFile, index=False)

                if index%20== 0:
                    sess= Restore_Session(sess)
            
            if index==end:
                break 
    
    except Exception as e:
        print("An exception occurred")
        print(e)
            
    closeSession(sess)
    # Save the outputs to csv
    outputFile= "testCheckPoints/output_{}.csv".format(index)
    df = pd.DataFrame({ 'review': review, 'predictedReview': predictedReview, 'sentiment': sentiment, 'ActualSentiment': ActualSentiment})
    df.to_csv(outputFile, index=False)


# Main Program that run the testing 

In [ ]:
file_path= "/kaggle/input/notebook867863cc1c/testReview.csv"
df = pd.read_csv(file_path, header=None)
# delete the zeroth row with null values
df.columns = ['index', 'text', 'sentiments']
df = df.drop([0])
print(df)
RunOnDataset(df)